# Problem Statement

A US bike-sharing provider BoomBikes has recently suffered considerable dips in their revenues. They have contracted a consulting company to understand the factors on which the demand for these shared bikes depends. Specifically, they want to understand the factors affecting the demand for these shared bikes in the American market. The company wants to know:

•	Which variables are significant in predicting the demand for shared bikes.

•	How well those variables describe the bike demands

You are required to model the demand for shared bikes with the available independent variables. It will be used by the management to understand how exactly the demands vary with different features. They can accordingly manipulate the business strategy to meet the demand levels and meet the customer's expectations. Further, the model will be a good way for management to understand the demand dynamics of a new market.


### Importing the necessary libraries 

In [813]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Step 1 - Reading and Exploring the dataset
Performing basic operations to get uber level understanding of the dataset


In [815]:
bike_df = pd.read_csv("../input/dataset/day.csv")
bike_df.head()

In [816]:
# The dataset has 730 rows and 16 columns 
bike_df.shape

In [817]:
# Looking at the unique number of values in each column
bike_df.nunique() 

In [818]:
bike_df.dtypes

In [819]:
bike_df.describe()

Substituting the values of the columns in some of the categorical variable with appropriate tags as mentioned in the data dictionary for better interpretability 
Some of the columns which are having only binary values such as 0 and 1 will not be altered because therequirement is already satisfied with the existing data type. 

In [820]:
bike_df['season'] = bike_df['season'].map({1:'Spring',2:'Summer',3:'Fall',4:'Winter'})
bike_df['mnth'] = bike_df['mnth'].map({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'})
bike_df['weathersit'] = bike_df['weathersit'].map({1:'Partly cloudy',2:'Mist',3:'Light Rain + Scattered clouds',4:'Fog'})
bike_df['weekday']  = bike_df['weekday'].map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})
bike_df.head(10)

In [821]:
#Checking for changed datatypes
bike_df.info()

### Step 2 - Data Vizualizations
#### Detecting the outliers using the box plots for each column with 'cnt' as the parameter 

In [822]:
columns = ['yr','holiday','workingday','season','weathersit','mnth']
for x in bike_df[columns]:
    sns.boxplot(x= bike_df[x],y=bike_df['cnt'])
    plt.show()

**Inferences from the above box plots**
* The number of users was high in 2019 comapred to 2018
* People tend to use more bikes on days which are not holidays
* The median of bike usage on non-working days and working days is almost the same
* Fall Season has the most number of users
* When the weather is partly cloudy, the bike usage will be hghest
* The Median number of users is highest in July

#### Vizualising the correlation of different variables using the heatmap
The correlation matrix vizualised using the heat map gives us an estimate of the dependencies of variables amongst themselevs. It can be observed from the below snippet that there is existance of large dependencies of some variables. 

In [823]:
plt.figure(figsize= (20,8))
sns.heatmap(bike_df.corr(),annot = True)

* The target variable 'cnt' is closely associated with the columns 'registered' and 'casual'. This observation is justified since 'cnt' is the sum of 'registered' and 'casual'
* The co-eff score with of 'cnt' is high for atemp and temp as well. It can be explored while calculating the VIF
* the column 'yr'also has good corelation with the 'cnt' column
* It can be seen that temp and atemp are very highly co-related, indiactes the presence of multicollinearity. 

#### Understanding the liner relationships of different variables using a pair plot

In [824]:
sns.pairplot(bike_df,vars = ['cnt', 'temp', 'atemp', 'hum','windspeed'])
plt.show()

All the numeric variables seem to have scattered data points, and a possibilty for fitting a line is observed

### Step 3 - Data Preperation
#### Creating dummy variables for category type columns

In [825]:
for i in ['season','mnth','weekday','weathersit']:
    i_df = pd.get_dummies(bike_df[i],drop_first = True,prefix = i)
    bike_df =pd.concat( [bike_df,i_df],axis =1)
    bike_df.drop([i],axis = 1,inplace = True)

In [826]:
bike_df.head(10)

#### Droping the variables that are not appropriate for building the model
* instant - This columns is an index and will not add any value for predicting the target variable
* date - Since there are month and year columns, this variable will not required
* casual,registered -  The sum of casual and registered equals to the value in the 'cnt' variable and it alone can compensate for the casual and registered columns. Hence these two variables can be dropped 

In [827]:
bike_df.drop(['instant','dteday','casual','registered'],axis = 1,inplace = True)

## Step 4 - Splitting the data into test and train datasets
A ratio of 80% to 20% is considered for train and test size of the dataset with a random state of 100

In [828]:
from sklearn.model_selection import train_test_split
np.random.seed(0)
df_train, df_test = train_test_split(bike_df, train_size = 0.8, test_size = 0.2, random_state = 100)
print(df_train.shape,'\n',df_test.shape )

There are 510 rows in the train dataset and 219 rows in the test set 

## Step 5 - Feature Scaling: Normalising the train set using MinMax Scaler

In [829]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
columns = ['temp','atemp','hum','windspeed','cnt']
df_train[columns] = scaler.fit_transform(df_train[columns])

In [830]:
df_train.head(10)

As seen from the result below the min value and max value for all the numeric columns are 0 and 1 respectively

In [831]:
df_train.describe()

Plotting the heat map again to check the dependencies in the train dataset after normalizing

In [832]:
plt.figure(figsize = (30,20))
sns.heatmap(df_train.corr(),annot = True)

The above graph gives a detailed description of the correlations of the variables in the train dataset. However looking at the relationships of the other variables with the 'cnt' variable is important 

In [833]:
plt.figure(figsize = (5,10))
sns.heatmap(pd.DataFrame(df_train.corr()['cnt']).sort_values(by= 'cnt',ascending = False),annot = True)

Considering the variable above 0.5 as significant values
* atemp,temp and yr on the positive side and season spring on the negative side have the highest corelations with the target variable.

## Step 6 - Building the Linear regression model
Splitting the train_df into x_train and y_train datsets

In [834]:
y_train = df_train.pop('cnt')
x_train = df_train
y_train.shape

In [835]:
print(x_train.shape)
x_train.head(5)

#### There are  two approaches to feed the variable to the model, which are explained as follows

* Backward selection - Adding all the variables to the model,and removing them sequetially after checkin the VIF and p-value

Initially we will start off with forward selection and move onto backward selection

* Forward selection- Adding the variables to the model to check if the adjusted R^2 increases.Starting with the highest co-related value to the target variable 'cnt'

In [836]:
# The stats model does not a constant or intercept value while fitting a line and by default the line will pass through the origin. 
# Therefor we have to add the constant for the variables that are fed into model
x_train_const = sm.add_constant(x_train[['atemp']])

In [837]:
#Building the model
lr = sm.OLS(y_train, x_train_const).fit()
lr.params

#### Plotting a scatter plot and line plot to vizualize the predictions and the line

In [838]:
plt.scatter(x_train_const.iloc[:,1],y_train)
plt.plot(x_train_const.iloc[:, 1], 0.162924 + 0.683633*x_train_const.iloc[:, 1], 'r')

In [839]:
print(lr.summary())

From the summary chart it can be seen that the variable 'atemp' has a 0.000 as he p-value indicating it has high significance to the model. But the R2 value is below 50%. 
Let us add the 'yr' variable and check if the R2 value increases

In [840]:
x_train_const = sm.add_constant(x_train[['atemp','yr']])
lr= sm.OLS(y_train,x_train_const).fit()
lr.params

In [841]:
print(lr.summary())

With 'yr' variable added to the mix, the R2 value shot up to almost 70%. 

In [842]:
x_train_const = sm.add_constant(x_train[['atemp','yr','season_Spring']])
lr= sm.OLS(y_train,x_train_const).fit()
lr.params

In [843]:
print(lr.summary())

When we add the 'season_spring' into the model, the R2 value increased by only 4%. Indicating further addition of any variables will only increase the effectiveness of the model marginally. We can test the other methods to explore the best set of variable to be incorporated for building the model

### Adding all the variables
#### Backward Selection

Starting with using RFE for selecting the best set of variables and then proceeding to elimate the variables as per the p-value and VIF to obtain an optimized set


Multicollinearity is also an importatnt factor that has to be considered for building a good model. Their presence can be detected with the help of VIF. We can use the VIF values for our advantage to remove the model using a quantitative approach.
All the variable which are having a VIF approximately more than 5 will be omitted. Since the VIF values change after each varible is removed, we have to follow an iterative method in order to arrive at the best set of variables.

In [844]:
# Running RFE with the output number of the variable equal to 13
lm = LinearRegression()
lm.fit(x_train, y_train)

rfe = RFE(lm, 15)             # running RFE
rfe = rfe.fit(x_train, y_train)

In [845]:
#Columns selected by RFE and their weights
list(zip(x_train.columns,rfe.support_,rfe.ranking_))

In [846]:
# Considering the columns which has 1 as the rfe ranking
rfe_cols = x_train.columns[rfe.support_]
rfe_cols

In [847]:
#Omitting the following variables
x_train.columns[~rfe.support_]

In [848]:
#Creating a new x_train dataset which only has the columns as suggested by the rfe model
x_train_rfe = x_train[rfe_cols]

#### Model 1

In [849]:
#Adding the constant and building the first model 
x_train_const = sm.add_constant(x_train_rfe)
lr_m1 = sm.OLS(y_train,x_train_const.astype(float)).fit()
lr_m1.params

In [850]:
print(lr_m1.summary())

In [851]:
# Looking at the VIF for the selected variables
vif = pd.DataFrame()
vif['Features'] = x_train_rfe.columns
vif['VIF'] = [variance_inflation_factor(x_train_rfe.values, i) for i in range(x_train_rfe.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [852]:
# Eliminating the 'hum' variable which has the highest VIF
x_train_rfe_m1 = x_train_rfe.drop(['hum'],axis = 1)

## Model 2

In [853]:
x_train_const = sm.add_constant(x_train_rfe_m1)
lr_m2 = sm.OLS(y_train,x_train_const.astype(float)).fit()
lr_m2.params

In [854]:
print(lr_m2.summary())

In [855]:
# Looking at the VIF for the selected variables
vif = pd.DataFrame()
vif['Features'] = x_train_rfe_m1.columns
vif['VIF'] = [variance_inflation_factor(x_train_rfe_m1.values, i) for i in range(x_train_rfe_m1.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

Although the variables 'temp' and 'weathersit_Partly cloudy' have high VIF, it is observed from the 
correlation plot that these variabels have high association with the 'cnt' target variable. Therefore
proceeding to eliminate the variable that has the next highest VIF

In [856]:
x_train_rfe_m2 = x_train_rfe_m1.drop(['weathersit_Mist'],axis = 1)

### Model 3

In [857]:
x_train_const = sm.add_constant(x_train_rfe_m2)
lr_m3 = sm.OLS(y_train,x_train_const.astype(float)).fit()
lr_m3.params

In [858]:
print(lr_m3.summary())

In [859]:
# Looking at the VIF for the selected variables
vif = pd.DataFrame()
vif['Features'] = x_train_rfe_m2.columns
vif['VIF'] = [variance_inflation_factor(x_train_rfe_m2.values, i) for i in range(x_train_rfe_m2.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

##### The p-values for the all the variables used for buliding the model 3 are almost equal to 0 upto three decimal places and the VIF is in the marginal limit. In order to further optimise the variable sets we use another RFE model on the list of columns we have obtained above. 

## Model 4

In [860]:
# Running RFE with the output number of the variable equal to 10

rfe = RFE(lm, 10)             # running RFE
rfe = rfe.fit(x_train_rfe_m2, y_train)

list(zip(x_train_rfe_m2.columns,rfe.support_,rfe.ranking_))
rfe_cols_model3 = x_train_rfe_m2.columns[rfe.support_]
rfe_cols_model3
x_train_rfe_m3 = x_train[rfe_cols_model3]

In [861]:
x_train_const = sm.add_constant(x_train_rfe_m3)
lr_m4 = sm.OLS(y_train,x_train_const).fit()
lr_m4.params

In [862]:
print(lr_m4.summary())

In [863]:
vif = pd.DataFrame()
vif['Features'] = x_train_rfe_m3.columns
vif['VIF'] = [variance_inflation_factor(x_train_rfe_m3.values, i) for i in range(x_train_rfe_m3.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

We can see from the above table that the VIF values for the varibale have further decreased and are eligible for consideration. The R2 value is around 0.83

#### Multiple iterations as seen below were experimented with, but the above approach was sufficient to arrive at the required accuracy

### Model 4

In [864]:
# x_train_const = sm.add_constant(x_train_rfe_m3)
# lr_m4 = sm.OLS(y_train,x_train_const.astype(float)).fit()
# lr_m4.params

In [865]:
# print(lr_m4.summary())

In [866]:
# vif = pd.DataFrame()
# vif['Features'] = x_train_rfe_m3.columns
# vif['VIF'] = [variance_inflation_factor(x_train_rfe_m3.values, i) for i in range(x_train_rfe_m3.shape[1])]
# vif['VIF'] = round(vif['VIF'], 2)
# vif = vif.sort_values(by = "VIF", ascending = False)
# vif

In [867]:
# x_train_rfe_m4 = x_train_rfe_m3.drop(['weekday_Sunday'],axis = 1)

### Model 5

In [868]:
# x_train_const = sm.add_constant(x_train_rfe_m4)
# lr_m5 = sm.OLS(y_train,x_train_const.astype(float)).fit()
# lr_m5.params

In [869]:
# print(lr_m5.summary())

In [870]:
# vif = pd.DataFrame()
# vif['Features'] = x_train_rfe_m4.columns
# vif['VIF'] = [variance_inflation_factor(x_train_rfe_m4.values, i) for i in range(x_train_rfe_m4.shape[1])]
# vif['VIF'] = round(vif['VIF'], 2)
# vif = vif.sort_values(by = "VIF", ascending = False)
# vif

In [871]:
# The mnth_Jul has high p-value. Removing it from the dataset
# x_train_rfe_m5 = x_train_rfe_m4.drop(['mnth_Jul'],axis = 1)

### Model 6



In [872]:
# x_train_const = sm.add_constant(x_train_rfe_m5)
# lr_m6 = sm.OLS(y_train,x_train_const.astype(float)).fit()
# lr_m6.params

In [873]:
# print(lr_m6.summary())

In [874]:
# vif = pd.DataFrame()
# vif['Features'] = x_train_rfe_m5.columns
# vif['VIF'] = [variance_inflation_factor(x_train_rfe_m5.values, i) for i in range(x_train_rfe_m5.shape[1])]
# vif['VIF'] = round(vif['VIF'], 2)
# vif = vif.sort_values(by = "VIF", ascending = False)
# vif

### Model 7

In [875]:
# x_train_const = sm.add_constant(x_train_rfe_m6)
# lr_m7 = sm.OLS(y_train,x_train_const.astype(float)).fit()
# lr_m7.params

In [876]:
# print(lr_m7.summary())

In [877]:
# vif = pd.DataFrame()
# vif['Features'] = x_train_rfe_m6.columns
# vif['VIF'] = [variance_inflation_factor(x_train_rfe_m6.values, i) for i in range(x_train_rfe_m6.shape[1])]
# vif['VIF'] = round(vif['VIF'], 2)
# vif = vif.sort_values(by = "VIF", ascending = False)
# vif

In [878]:
# y_train.head(5)

## Step 7 - Residual Analysis

Plotting a histogram to see if the error terms are normally distributes in order to confirm the assumption of the linear regression model

In [879]:
#Predicting the dependent variables using the model the model that was built above
y_train_cnt = lr_m4.predict(x_train_const)
y_train_cnt.shape

In [880]:
fig = plt.figure()
sns.distplot((y_train - y_train_cnt), bins = 20)
fig.suptitle('Error Terms', fontsize = 20)# Plot heading 
plt.xlabel('Errors', fontsize = 18)     

It can be seen above that the error terms are normally distributed and adheres to the assumptions of linear regression

In [881]:
# Calculatng the R2 value of the train dataset
r2_score(y_train,y_train_cnt)

## Step 8 - Predecting the values using the test dataset

### Normalizing the test dataset

In [882]:
#Normalizing the numeric columns of the test dataset
columns = ['temp','atemp','hum','windspeed','cnt']
df_test[columns] = scaler.fit_transform(df_test[columns])

In [883]:
#Seperating the x_test and y_test from the df_test dataset
y_test = df_test.pop('cnt')
x_test = df_test

In [884]:
#Adding a constant
x_test_const = sm.add_constant(x_test)

In [885]:
# Applying the final model to the test dataset 
cols = x_train_const.columns.tolist()
x_test_const_mod = x_test_const[cols]
y_test_pred = lr_m4.predict(x_test_const_mod)

In [886]:
x_test_const_mod.info()

## Step 9 - Model Evaluation

In [887]:
# Plotting y_test and y_pred to understand the spread

fig = plt.figure()
plt.scatter(y_test, y_test_pred)
fig.suptitle('y_test vs y_test_pred', fontsize = 20)         # Plot heading 
plt.xlabel('y_test', fontsize = 18)                          # X-label
plt.ylabel('y_test_pred', fontsize = 16)  

 ## Printing the final set of 10 variables and their respective co-efficients which affect the demand generated for the Bike sharing company

In [888]:
pd.DataFrame(lr_m4.params,columns= ['Co-efficient']).rename_axis('Variable_Name').reset_index().sort_values(by='Co-efficient',ascending = False)

In [889]:
#Calculating the R2 value of the final model created for the test dataset
r2_score(y_test,y_test_pred)

## Final Observations
### The equation of the MLR model will be 
#### cnt = 0.108686 + 0.483505 * temp + 0.234174*yr + 0.094583 * weathersit_Partly cloudy + 0.067758 * weekday_Sunday + 0.056515 * workingday + 0.043058 * season_Winter - 0.08942 * mnth_Jul - 0.116136 * season_Spring - 0.173345 * windspeed - 0.041939 * mnth_Nov 

The weight that each variable imposses on the target variable can be understood from the above equation along with intercept or the co-efficent of the best fit line.
Findings from the equation
* Temprature (temp) has the highest positive influence on the target variable
* Windspeed has the highest negative influence on the 'cnt' target variable
* Winter season has the least influence on the target variable compared other variable under consideration

The R2 value for the teat dataset is 0.74822

#### Inferences from the model 

* Temprature plays a crucial role, as it can be seen from the model that as the temprature increases the number of people renting the bike also increases.
* Although workweek has some influence on the number of bikes that are hired by people, specific days of a workweek does not pose any dependency on the 'cnt' target variable 
* Sunday has high number of users compared to any aother days of the week
* Partly cloudy weather invites more of bike rentals 
* The seasons winter and spring have some impact on the target variable.
* In the month of July and November the people rent less bikes. Apart from this month none of the other months in an year bears any to least significance 
* If the wind speed increase the number of bike users decrease. It is the most detrimental factor out of the ones that are observed
* The Year variable has a positive co-efficient of 0.234 which implies that the count of bike hires has increased over the years 


The major indicators that influence demand are Temprature, Year and the Windspeed